<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Helper-functions" data-toc-modified-id="Helper-functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Helper functions</a></span></li><li><span><a href="#Paper-Surveys" data-toc-modified-id="Paper-Surveys-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Paper Surveys</a></span><ul class="toc-item"><li><span><a href="#First-paper-wave" data-toc-modified-id="First-paper-wave-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>First paper wave</a></span></li><li><span><a href="#Second-paper-wave" data-toc-modified-id="Second-paper-wave-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Second paper wave</a></span></li><li><span><a href="#Third-paper-wave" data-toc-modified-id="Third-paper-wave-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Third paper wave</a></span></li></ul></li><li><span><a href="#Online-Surveys" data-toc-modified-id="Online-Surveys-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Online Surveys</a></span><ul class="toc-item"><li><span><a href="#First-wave" data-toc-modified-id="First-wave-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>First wave</a></span></li><li><span><a href="#Second-waves" data-toc-modified-id="Second-waves-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Second waves</a></span></li><li><span><a href="#Third-waves" data-toc-modified-id="Third-waves-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Third waves</a></span></li></ul></li><li><span><a href="#Clean-up-dataframe" data-toc-modified-id="Clean-up-dataframe-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Clean up dataframe</a></span></li><li><span><a href="#Explore-Data" data-toc-modified-id="Explore-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Explore Data</a></span></li><li><span><a href="#Export-data" data-toc-modified-id="Export-data-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Export data</a></span></li></ul></div>

# survey_clean.py
This file takes the eight tax rebate survey files and combines them into one CSV.

In [ ]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
import datetime as dt

## Helper functions

In [ ]:
def get_receipt_date(df):
    """
    Returns `df` with a col "date_receipt" using
    cols `q7` and `q7a`.
    
    `date_receipt` = 2008-01-01 for unknown dates.
    """
    # create int cols
    df['month'] = df.q7.apply(lambda x: int(x) + 3 if x else 1)
    df['day'] = df.q7a.apply(lambda x: int(x) - 1 if x and int(x) > 1 else 1)
    
    # handle out-of-range dates
    df.loc[df.month.isin([4, 6]) & (df.day > 30), 'day'] = 30
    
    # calc date
    df['date_receipt'] = df.apply(lambda x: dt.date(2008, x.month if x.month else 1, x.day if x.day else 1), 1)
    
    # drop cols
    df.drop(['month', 'day'], 1, inplace=True)
                                  
    return df

## Paper Surveys

### First paper wave

In [ ]:
# import and clean survey

df = pd.read_csv('../data/raw_data/survey/n280532_taxrebate_ppr_wv1.txt', sep = '\t', header=None, dtype='str')
df.columns = ['hhid', 'date', 'q1', 'surveyn_p1_p2', 'q9q11', 'c6', 'c7']
df.head()

# examine entries with errors
display(df.loc[df.c6.notnull(), :])
display(df.loc[df.c7.notnull(), :])

# Drop all questions after q6 for those with errors
# then drop c6 and c7 (because all responses to q6 require follow up)
df.loc[df.c6.notnull(), 'q9q11'] = np.NaN
df.drop(['c6', 'c7'], 1, inplace=True)
df.head()

# examine
print(len(df))
print(len(df.hhid.unique()))
df.isnull().sum()

In [ ]:
# Separate surveyn and q9q11
df['surveyn'] = df.surveyn_p1_p2.str.slice(stop=6)
df['q2'] = df.surveyn_p1_p2.str.slice(start=6, stop=7)
df['q3'] = df.surveyn_p1_p2.str.slice(start=7, stop=8)
df['q4'] = df.surveyn_p1_p2.str.slice(start=8, stop=9)
df['q5'] = df.surveyn_p1_p2.str.slice(start=9, stop=10)
df['q6'] = df.surveyn_p1_p2.str.slice(start=10, stop=11)
df['q7'] = df.surveyn_p1_p2.str.slice(start=11, stop=12)
df['q7a'] = df.surveyn_p1_p2.str.slice(start=12, stop=14)
df['q8'] = df.surveyn_p1_p2.str.slice(start=14, stop=15)
df['q8a'] = df.surveyn_p1_p2.str.slice(start=15)

df['q9'] = df.q9q11.str.slice(stop=1)
df['q10'] = df.q9q11.str.slice(start=1, stop=2)
df['q11-1'] = df.q9q11.str.slice(start=2, stop=6)
df['q11-2'] = df.q9q11.str.slice(start=6, stop=10)
df['q11-3'] = df.q9q11.str.slice(start=10, stop=14)
df['q11-4'] = df.q9q11.str.slice(start=14, stop=18)
df['q11-5'] = df.q9q11.str.slice(start=18)

# drop redundant vars
df.drop(['surveyn_p1_p2', 'q9q11'], 1, inplace=True)

# add var to indicate the hhid answered the paper survey 1
df['ans_p1'] = 1

# date-ify the date
df['date_p1'] = df.date.apply(lambda x: dt.date(int('20' + x[:2]), int(x[2:4]), int(x[4:])))
df.drop('date', 1, inplace=True)

# get receipt date
df = get_receipt_date(df)

df.head()

### Second paper wave

In [ ]:
# get next paper survey

temp = pd.read_csv('../data/raw_data/survey/n280534_taxrebate_ppr_wv2.txt', sep = '\t', header=None, dtype='str')
temp.columns = ['hhid', 'date', 'surveyn', 'p2', 'q9q11', 'c']

# drop the one error
temp = temp[temp.c.isnull()]

# split the relevant questions
temp['q6'] = temp.p2.str.slice(stop=1)
temp['q7'] = temp.p2.str.slice(start=1, stop=2)
temp['q7a'] = temp.p2.str.slice(start=2, stop=4)
temp['q8'] = temp.p2.str.slice(start=4, stop=5)
temp['q8a'] = temp.p2.str.slice(start=5)

temp['q9'] = temp.q9q11.str.slice(stop=1)
temp['q10'] = temp.q9q11.str.slice(start=1, stop=2)
temp['q11-1'] = temp.q9q11.str.slice(start=2, stop=6)
temp['q11-2'] = temp.q9q11.str.slice(start=6, stop=10)
temp['q11-3'] = temp.q9q11.str.slice(start=10, stop=14)
temp['q11-4'] = temp.q9q11.str.slice(start=14, stop=18)
temp['q11-5'] = temp.q9q11.str.slice(start=18)

# drop unecessary vars
temp.drop(['p2', 'q9q11', 'c'], 1, inplace=True)

# add var to indicate the hhid answered the paper survey 1
temp['ans_p2'] = 1

# date-ify the date
temp['date_p2'] = temp.date.apply(lambda x: dt.date(int('20' + x[:2]), int(x[2:4]), int(x[4:])))
temp.drop('date', 1, inplace=True)

# get receipt date
temp = get_receipt_date(temp)

print(len(temp))
temp.head(3)

In [ ]:
df.head(3)

In [ ]:
# how many obs in df also in temp?
temp.loc[temp.hhid.isin(df.loc[df.q6 == "1"].hhid)]

In [ ]:
# merge with df
# keep most recent survey response for q6 onwards

# first, merge q1-q5 and ans_p1 from df into temp
temp = pd.merge(temp, df[\
        ['hhid','q1','q2','q3','q4','q5', 'ans_p1', 'date_p1']],
        how='left', on ='hhid')

# drop all observations from df found in temp
df = df[~df.hhid.isin(temp.hhid)]
print(len(temp))
print(len(df))

# concat df with temp
df = pd.concat([df, temp], 0, sort=True)
df.sort_values('hhid', inplace=True)

print(len(df))
print(len(df.hhid.unique()))
df.head()

### Third paper wave

In [ ]:
# last paper survey
temp = pd.read_csv('../data/raw_data/survey/n280599_taxrebate_ppr_wv2.txt', sep = '\t', header=None, dtype='str')
temp.columns = ['hhid', 'date', 'q1', 'surveyn_p1_p2', 'q9q11', 'c6', 'c7']

# investigate errors
display(temp.loc[(temp.c6.notnull()) | (temp.c7.notnull())])
# all have not yet received ESP
temp = temp[(temp.c7.isnull()) & (temp.c6.isnull())]

# split questions
temp['surveyn'] = temp.surveyn_p1_p2.str.slice(stop=6)
temp['q2'] = temp.surveyn_p1_p2.str.slice(start=6, stop=7)
temp['q3'] = temp.surveyn_p1_p2.str.slice(start=7, stop=8)
temp['q4'] = temp.surveyn_p1_p2.str.slice(start=8, stop=9)
temp['q5'] = temp.surveyn_p1_p2.str.slice(start=9, stop=10)
temp['q6'] = temp.surveyn_p1_p2.str.slice(start=10, stop=11)
temp['q7'] = temp.surveyn_p1_p2.str.slice(start=11, stop=12)
temp['q7a'] = temp.surveyn_p1_p2.str.slice(start=12, stop=14)
temp['q8'] = temp.surveyn_p1_p2.str.slice(start=14, stop=15)
temp['q8a'] = temp.surveyn_p1_p2.str.slice(start=15)

temp['q9'] = temp.q9q11.str.slice(stop=1)
temp['q10'] = temp.q9q11.str.slice(start=1, stop=2)
temp['q11-1'] = temp.q9q11.str.slice(start=2, stop=6)
temp['q11-2'] = temp.q9q11.str.slice(start=6, stop=10)
temp['q11-3'] = temp.q9q11.str.slice(start=10, stop=14)
temp['q11-4'] = temp.q9q11.str.slice(start=14, stop=18)
temp['q11-5'] = temp.q9q11.str.slice(start=18)

temp.drop(['surveyn_p1_p2', 'q9q11', 'c6', 'c7'], 1, inplace=True)

# date-ify the date
temp['date_p3'] = temp.date.apply(lambda x: dt.date(int('20' + x[:2]), int(x[2:4]), int(x[4:])))
temp.drop('date', 1, inplace=True)

# get receipt date
temp = get_receipt_date(temp)

print(len(temp))
temp.head()

In [ ]:
# Overlapping obs?

hhids = temp.loc[(temp.q6 == "1") & (temp.hhid.isin(df.loc[df.q6 == "1"].hhid))].hhid

cols = ['hhid', 'q6', 'date_receipt', 'q8a', 'q9', 'q10', 'q11-1']
temp2 = pd.merge(temp.loc[temp.hhid.isin(hhids), cols  + ['date_p3']],
                 df.loc[df.hhid.isin(hhids), cols + ['date_p1']], how='inner', on='hhid')
display(temp2.sort_values('hhid'))

# basically the same, keep the OG data where households should not have been sent survey again

In [ ]:
# merge with df

# remove overlapping records
# Gist: remove from temp hhids that have q6 = 1 (these households should not have
#  received second survey). Then keep second survey for repeats where follow-up
#  is appropriate.
print(sum(df.hhid.isin(temp.hhid)))
temp = temp.loc[~temp.hhid.isin(df.loc[df.q6 == 1].hhid)]
df = df.loc[~df.hhid.isin(temp.hhid)]

# concat new obs
df = pd.concat([df, temp], 0, sort=True)

# how many obs
print(len(df))
print(len(df.hhid.unique()))
df.head()

## Online Surveys

In [ ]:
# copy of df as dfpaper
df['paper'] = 1
dfpaper = df.copy()

### First wave

In [ ]:
def raw_to_clean(fname, p2only=False):
    """
    Takes a filename and returns a cleaned dataframe.
    """  
    df = temp = pd.read_csv(f'../data/raw_data/survey/{fname}', sep = '\t', header=None, dtype='str')
    if len(df.columns) == 3:
        df.columns = ['c1', 'c2', 'c3']
    elif len(df.columns) == 4:
        df.columns = ['c0', 'c01', 'c2', 'c3']
        df['c1'] = df.c0 + "0" + df.c01
        df.drop(['c0', 'c01'], 1, inplace=True)
    else:
        raise ValueError('Not 3 or 4 columns')
    
    df['hhid'] = df.c1.str.slice(stop=8)
    df['date'] = df.c1.str.slice(start=15, stop=21)
    df['surveyn'] = fname[1:7]
    
    if p2only == False:
        df['q1'] = df.c1.str.slice(start=27, stop=28)
        qlist1 = ['q2', 'q3', 'q4', 'q5', 'q6',\
         'q7', 'q7a', 'q8', 'q8a']
        lenlist1 = [0, 1, 2, 3, 4, 5, 6, 8, 9, 13]
        lenlist1 = [x+6 for x in lenlist1]
    else:
        qlist1 = ['q6', 'q7', 'q7a', 'q8', 'q8a']
        lenlist1 = [0, 1, 2, 4, 5, 9]
        
    qlist2 = ['q9', 'q10', 'q11-1', 'q11-2', \
              'q11-3', 'q11-4', 'q11-5']
    lenlist2 = [0, 1, 2, 6, 10, 14, 18, 22]
    for i in range(len(qlist1)):
        df[qlist1[i]] = df.c2.str.slice(start=lenlist1[i], stop=lenlist1[i+1])
    for i in range(len(qlist2)):
        df[qlist2[i]] = df.c3.str.slice(start=lenlist2[i], stop=lenlist2[i+1])
    df = df.drop(['c1', 'c2', 'c3'], 1)
    
    # date-ify dates
    df['date'] = df.date.apply(lambda x: dt.date(int('20' + x[:2]), int(x[2:4]), int(x[4:])))

    # get receipt date
    df = get_receipt_date(df)
    
    # print lengths
    print(len(df))
    print(len(df.hhid.unique()))
    print(sum(dfpaper.hhid.isin(df.hhid)))
        
    return df
    
# test
display(raw_to_clean('n280556_taxrebate_may08_wv1.txt').head())

In [ ]:
temp = raw_to_clean('n280556_taxrebate_may08_wv1.txt')
temp.rename(columns={'date': 'date_p1'}, inplace=True)
temp['ans_p1'] = 2

# merge with df 
df = pd.concat([df, temp], 0, sort=True)
df.head()

### Second waves

In [ ]:
temp = raw_to_clean('n280560_taxrebate_may08_wv2.txt', p2only=True)
temp.rename(columns={'date': 'date_p2'}, inplace=True)
temp['ans_p2'] = 2

# merge with df

# copy old info for repeat hhids to temp
print(sum(temp.hhid.isin(df.hhid)))
temp = pd.merge(temp, df[\
        ['hhid','q1','q2','q3','q4','q5']],
        how='left', on ='hhid')

# remove old hhids
df = df[~df.hhid.isin(temp.hhid)]

# concat both dfs
df = pd.concat([df, temp], 0, sort=True)
df.sort_values('hhid', inplace=True)

df.head()

In [ ]:
temp = raw_to_clean('n280683_taxrebate_may08_wv2.txt', p2only=False)
temp.rename(columns={'date': 'date_p2'}, inplace=True)
temp['ans_p2'] = 3
print(sum(temp.hhid.isin(df.hhid)))

# merge with df

# remove old hhids
df = df[~df.hhid.isin(temp.hhid)]

# concat both dfs
df = pd.concat([df, temp], 0, sort=True)
df.sort_values('hhid', inplace=True)

df.head()

### Third waves

In [ ]:
temp = raw_to_clean('n280756_taxrebate_may08_wv3.txt', p2only=False)
temp.rename(columns={'date': 'date_p3'}, inplace=True)
temp['ans_p3'] = 2
print(sum(temp.hhid.isin(df.hhid)))

# merge with df

# remove old hhids
df = df[~df.hhid.isin(temp.hhid)]

# concat both dfs
df = pd.concat([df, temp], 0, sort=True)
df.sort_values('hhid', inplace=True)

df.head()

In [ ]:
temp = raw_to_clean('n280757_taxrebate_may08_wv3.txt', p2only=True)
temp.rename(columns={'date': 'date_p3'}, inplace=True)
temp['ans_p3'] = 3

# merge with df

# copy old info for repeat hhids to temp
print(sum(temp.hhid.isin(df.hhid)))
temp = pd.merge(temp, df[\
        ['hhid','q1','q2','q3','q4','q5']],
        how='left', on ='hhid')

# remove old hhids
df = df[~df.hhid.isin(temp.hhid)]

# concat both dfs
df = pd.concat([df, temp], 0, sort=True)
df.sort_values('hhid', inplace=True)

df.head()

In [ ]:
# get total observations

print(len(df.hhid.unique()))
print(len(df))
print(df.q6.value_counts())
print(df.notnull().sum())

In [ ]:
df.surveyn.value_counts().head(10)

## Clean up dataframe

In [ ]:
# keep only those who receive a check

print(df.q6.value_counts())
print(sum(df.q6 != "1") / len(df))
dfc = df.loc[df.q6 == "1"]

# keep only those who answer both parts of the survey
print(sum(dfc.q1.isnull()) / len(df))
dfc = dfc.loc[dfc.q1.notnull()]

# keep only those who provide a complete date of receipt
print(sum(dfc.q7a == "01") / len(df))
dfc = dfc.loc[dfc.q7a != "01"]

print(len(dfc))

In [ ]:
dfc.isnull().sum()

In [ ]:
# fill missings
for c in ['ans_p1', 'ans_p2', 'ans_p3']:
    dfc.loc[dfc[c].isnull() | (dfc[c] == ''), c] = 0
    dfc[c] = np.array(dfc[c], dtype='int')
    
# edit missings
for c in ['q8a', 'q11-1', 'q11-2', 'q11-3', 'q11-4', 'q11-5']:
    dfc.loc[dfc[c].isin(['dnk', '']), c] = 0
dfc.loc[dfc.paper.isnull(), 'paper'] = 0

# int types
for c in ['q1', 'q10', 'q11-1', 'q11-2', 'q11-3', 'q11-4',
       'q11-5', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q7a', 'q8', 'q8a', 'q9',
       'surveyn', 'hhid', 'paper']:
    print(c)
    dfc[c] = np.array(dfc[c], dtype='int')

In [ ]:
dfc.dtypes

In [ ]:
# make same cuts as Broda & Parker 2014
print(len(dfc))

# already dropped households reporting not receiving an ESP and no date

# First, drop reporting receipt date after answering earlier survey.
#  Allow grace period in case survey results recording arriving later.

# case1: date_receipt before date_p1 and one of the two following surveys answered
grace_period = dt.timedelta(days=2)
dfc['dropx'] = np.array((dfc.date_receipt + grace_period < dfc.date_p1) & 
                               dfc.date_p1.notnull() & 
                            ((dfc.date_p2.notnull() & 
                               (dfc.ans_p3 == 0)) |
                            dfc.date_p3.notnull()), dtype='int')
print(dfc.dropx.value_counts())
# case2: date_receipt before date_p2 and date_p3 answered
dfc['dropx'] += np.array((dfc.date_receipt + grace_period < dfc.date_p2) & 
                               dfc.date_p2.notnull() & 
                               dfc.date_p3.notnull(), dtype='int')
print(dfc.dropx.value_counts())

# Next, drop those reporting receipt after submitting survey
dfc['date_final_survey'] = dfc.date_p3
dfc.loc[dfc.date_final_survey.isnull(), 'date_final_survey'] = dfc.loc[dfc.date_final_survey.isnull()].date_p2
dfc.loc[dfc.date_final_survey.isnull(), 'date_final_survey'] = dfc.loc[dfc.date_final_survey.isnull()].date_p1
print(sum(dfc.date_final_survey.isnull()))
dfc['dropx'] += np.array((dfc.date_receipt - grace_period > dfc.date_final_survey), dtype='int')
print(dfc.dropx.value_counts())

# Next, drop those reporting receiving direct deposit or mail outside of acceptable range
# For mail, allow extra 7 days for mail time
# Mail
grace_period = dt.timedelta(days=7)
start_date =  dt.date(2008, 5, 16)
end_date = dt.date(2008, 7, 11)
dfc['dropx'] += np.array(~(dfc.date_receipt.between(start_date - grace_period, end_date + 2 * grace_period)) &
                          (dfc.q8 == 2), dtype='int')
print(dfc.dropx.value_counts())

# Direct deposit
start_date =  dt.date(2008, 4, 28)  # direct deposit
end_date = dt.date(2008, 5, 16)  # direct deposit
dfc['dropx'] += np.array(~(dfc.date_receipt.between(start_date - grace_period, end_date + 1 * grace_period)) &
                          (dfc.q8 != 2), dtype='int')
print(dfc.dropx.value_counts())

In [ ]:
# drop where necessary

dfc = dfc.loc[dfc.dropx == 0]
dfc.drop('dropx', 1, inplace=True)
dfc.head()

## Explore Data

In [ ]:
dfc.head(1)

In [ ]:
# sanity checks
for i in ['q1', 'q2', 'q3', 'q4', 'q5', 'q6',\
         'q7', 'q7a', 'q8', 'q9', 'q10']:
    print('\n' + i)
    display(dfc.loc[:, i].value_counts().sort_index())
    
for i in ['q8a', 'q11-1', 'q11-2', 'q11-3', 'q11-4', 'q11-5']:
    print('\n' + i)
    display(dfc.loc[:, i].value_counts())

In [ ]:
pd.crosstab(dfc.q3, dfc.q4, margins=True)
# q3 = 1 -> spend today
# q3 = 2 -> save
# q4 = 1 -> 2 months saved
# q4 = 2 -> <2 months saved

# 1 - 2 -> hand to mouth
# 2 - 1 -> successful planners
# 2 - 2 -> liars (unsuccessful planners)
# 1 - 1 -> retirees 

In [ ]:
# q6 == 1 -> received rebate
display(dfc.q6.value_counts(dropna=False))

In [ ]:
# q7 = month received
display(dfc.q7.value_counts(dropna=False))

In [ ]:
# q7a = day received
# off by 1, 01 == IDK
display(dfc.q7a.value_counts(dropna=False).sort_index())

In [ ]:
# q8a = rebate amount
display(dfc.q8a.value_counts().head(20))

In [ ]:
# q9 = amount expecting
dfc.q9.value_counts()

In [ ]:
dfc.head()

## Export data

In [ ]:
dfc.to_csv('../data/gen_data/survey_cleaned.csv', index=False)